### CV2 Manual Labeling with SAM implement

In [8]:
import sys
sys.path.append('/Users/steeeve/Documents/csiss/streetview_crop_classification/cropClassification')
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
import numpy as np

from sam_functions import *


In [ ]:
np.random.seed(42)
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
# print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs 
    # (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )
# print(bool(os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]))

In [10]:
df_path = "/Users/steeeve/Documents/csiss/data/iowa_nebraska_labeled_imgs_filtered_reclass.json"
mask_save_path = "/Users/steeeve/Documents/csiss/data/mask_files"
model_chkpt = "/Users/steeeve/Documents/csiss/segment-anything-2/checkpoints/sam2_hiera_large.pt"
model_cfg = "sam2_hiera_l.yaml"
mask_gdf_path = "/Users/steeeve/Documents/csiss/data/mask_files/valid_img_mask.geojson"
mask_fld_path = "/Users/steeeve/Documents/csiss/data/mask_files"

In [ ]:
gdf = gpd.read_file(df_path, driver="GEOJSON")
gdf_crop = gdf.loc[gdf.crop_type != "Other"].reset_index(drop=True)
gdf_crop.head()

In [12]:
# sam2_manual_masking(gdf_crop[:40], [model_chkpt, model_cfg, device], mask_save_path)

In [ ]:
mask_gdf = gpd.read_file(mask_gdf_path, driver="GeoJSON")
mask_gdf.head()

In [ ]:
# %matplotlib inline
evaluate_img_mask_pair(mask_gdf, mask_fld_path)